In [1]:
import numpy as np
import pandas as pd

In [15]:
df = pd.read_csv('../data/daily_transport_rail_with_special_day.csv')

In [16]:
df.isna().sum()

วันที่             0
day_name           0
day_type           0
holiday_type       0
holiday_subject    0
ยานพาหนะ/ท่า       0
ปริมาณ             1
dtype: int64

In [17]:
# หาและแสดงแถวที่มีค่า NaN ในคอลัมน์ 'ปริมาณ'
nan_rows = df[df['ปริมาณ'].isna()]
display(nan_rows)  # แสดงรายละเอียดของแถวที่มีค่าว่าง

# ถ้าต้องการ index ของแถวที่มี NaN
print("Index ของแถวที่มี NaN:", nan_rows.index.tolist())

# เติมค่าเฉลี่ยของ ARL ต่อเดือนในแถวที่มีค่า NaN
# สร้างคอลัมน์ month จากวันที่
import pandas as pd
if 'month' not in df.columns:
    df['month'] = pd.to_datetime(df['วันที่']).dt.month

# คำนวณค่าเฉลี่ยปริมาณการใช้งาน ARL ต่อเดือน
arl_monthly_mean = df[df['ยานพาหนะ/ท่า'] == 'รถไฟฟ้า ARL'].groupby('month')['ปริมาณ'].mean()

# เติมค่า NaN สำหรับ ARL โดยใช้ค่าเฉลี่ยของเดือนนั้น
mask = (df['ยานพาหนะ/ท่า'] == 'รถไฟฟ้า ARL') & (df['ปริมาณ'].isna())
df.loc[mask, 'ปริมาณ'] = df.loc[mask, 'month'].map(arl_monthly_mean)

# ตรวจสอบผลลัพธ์หลังเติม
filled_rows = df[mask]
display(filled_rows)

,วันที่,day_name,day_type,holiday_type,holiday_subject,ยานพาหนะ/ท่า,ปริมาณ
9586,2025-01-20,วันจันทร์,1,normal,normal,รถไฟฟ้า ARL,NaN


Index ของแถวที่มี NaN: [9586]


,วันที่,day_name,day_type,holiday_type,holiday_subject,ยานพาหนะ/ท่า,ปริมาณ,month
9586,2025-01-20,วันจันทร์,1,normal,normal,รถไฟฟ้า ARL,52474.135135,1


In [20]:
df[df['ปริมาณ'] == 0]

,วันที่,day_name,day_type,holiday_type,holiday_subject,ยานพาหนะ/ท่า,ปริมาณ,month
6952,2024-01-03,วันพุธ,1,normal,normal,รถไฟฟ้าสายสีเหลือง,0.0,1
8811,2024-10-01,วันอังคาร,1,normal,normal,รถไฟฟ้าสายสีแดง,0.0,10
9180,2024-11-23,วันเสาร์,0,normal,normal,รถไฟฟ้า ARL,0.0,11
9271,2024-12-06,วันศุกร์,1,normal,normal,รถไฟฟ้า ARL,0.0,12
9299,2024-12-10,วันอังคาร,0,holiday,วันรัฐธรรมนูญ,รถไฟฟ้า ARL,0.0,12
9558,2025-01-16,วันพหัสบดี,1,normal,normal,รถไฟฟ้า ARL,0.0,1
9873,2025-03-02,วันอาทิตย์,0,normal,normal,รถไฟฟ้า ARL,0.0,3
10060,2025-03-29,วันเสาร์,0,normal,normal,รถไฟฟ้าสายสีเหลือง,0.0,3
10061,2025-03-29,วันเสาร์,0,normal,normal,รถไฟฟ้าสายสีชมพู,0.0,3
10068,2025-03-30,วันอาทิตย์,0,normal,normal,รถไฟฟ้าสายสีชมพู,0.0,3


In [21]:
# เติมค่า 0 ด้วยค่าเฉลี่ยรายเดือนตามแต่ละยานพาหนะ
if 'month' not in df.columns:
    df['month'] = pd.to_datetime(df['วันที่']).dt.month
# คำนวณค่าเฉลี่ยปริมาณต่อเดือนต่อยานพาหนะ (ไม่รวมค่า 0)
mode_month_mean = df[df['ปริมาณ'] > 0].groupby(['month', 'ยานพาหนะ/ท่า'])['ปริมาณ'].mean()
# เติมค่า 0 ด้วยค่าเฉลี่ยรายเดือนตามโหมด
mask_zero = df['ปริมาณ'] == 0
df.loc[mask_zero, 'ปริมาณ'] = df.loc[mask_zero].apply(
    lambda row: mode_month_mean.get((row['month'], row['ยานพาหนะ/ท่า']), 0), axis=1
)

# ย้ายข้อมูลวันเดียวกันให้อยู่ในแถวเดียวกัน (pivot)
# ใช้ pivot_table เพื่อรวบรวมปริมาณตามยานพาหนะต่อวัน
df_pivot = df.pivot_table(
    index=['วันที่', 'day_name', 'day_type', 'holiday_type', 'holiday_subject'],
    columns='ยานพาหนะ/ท่า',
    values='ปริมาณ',
    aggfunc='sum',
    fill_value=0
).reset_index()

# แสดงผลลัพธ์ตัวอย่าง
display(df_pivot.head())

ยานพาหนะ/ท่า,วันที่,day_name,day_type,holiday_type,holiday_subject,รถไฟฟ้า ARL,รถไฟฟ้า BTS,รถไฟฟ้าสายสีชมพู,รถไฟฟ้าสายสีน้ำเงิน,รถไฟฟ้าสายสีม่วง,รถไฟฟ้าสายสีเหลือง,รถไฟฟ้าสายสีแดง
0,2020-01-01,วันพุธ,0,holiday,วันขึ้นปีใหม่,48790.0,461400.0,0.0,236053.0,23135.0,0.0,0.0
1,2020-01-02,วันพหัสบดี,1,normal,normal,65090.0,720500.0,0.0,341059.0,39445.0,0.0,0.0
2,2020-01-03,วันศุกร์,1,normal,normal,69465.0,782100.0,0.0,373005.0,43185.0,0.0,0.0
3,2020-01-04,วันเสาร์,0,normal,normal,53304.0,574400.0,0.0,274151.0,30550.0,0.0,0.0
4,2020-01-05,วันอาทิตย์,0,normal,normal,50992.0,503000.0,0.0,237162.0,26012.0,0.0,0.0


In [23]:
# ปัดเศษค่าตัวเลขทั้งหมดเป็นจำนวนเต็ม
numeric_cols = df_pivot.select_dtypes(include=['number']).columns
if len(numeric_cols) > 0:
    df_pivot[numeric_cols] = df_pivot[numeric_cols].round().astype(int)

# แสดงตัวอย่างหลัง rounding
display(df_pivot.head())

ยานพาหนะ/ท่า,วันที่,day_name,day_type,holiday_type,holiday_subject,รถไฟฟ้า ARL,รถไฟฟ้า BTS,รถไฟฟ้าสายสีชมพู,รถไฟฟ้าสายสีน้ำเงิน,รถไฟฟ้าสายสีม่วง,รถไฟฟ้าสายสีเหลือง,รถไฟฟ้าสายสีแดง
0,2020-01-01,วันพุธ,0,holiday,วันขึ้นปีใหม่,48790,461400,0,236053,23135,0,0
1,2020-01-02,วันพหัสบดี,1,normal,normal,65090,720500,0,341059,39445,0,0
2,2020-01-03,วันศุกร์,1,normal,normal,69465,782100,0,373005,43185,0,0
3,2020-01-04,วันเสาร์,0,normal,normal,53304,574400,0,274151,30550,0,0
4,2020-01-05,วันอาทิตย์,0,normal,normal,50992,503000,0,237162,26012,0,0


In [24]:
# แทนค่า 0 ด้วย 'not available' ในคอลัมน์ปริมาณหลัง rounding
# เลือกคอลัมน์ปริมาณรถไฟฟ้าโดยไม่รวมคอลัมน์ดัชนี
passenger_cols = [col for col in df_pivot.columns if col not in ['วันที่','day_name','day_type','holiday_type','holiday_subject']]
df_pivot[passenger_cols] = df_pivot[passenger_cols].replace(0, 'not available')
# แสดงตัวอย่างหลังแทนค่า
display(df_pivot.head())

ยานพาหนะ/ท่า,วันที่,day_name,day_type,holiday_type,holiday_subject,รถไฟฟ้า ARL,รถไฟฟ้า BTS,รถไฟฟ้าสายสีชมพู,รถไฟฟ้าสายสีน้ำเงิน,รถไฟฟ้าสายสีม่วง,รถไฟฟ้าสายสีเหลือง,รถไฟฟ้าสายสีแดง
0,2020-01-01,วันพุธ,0,holiday,วันขึ้นปีใหม่,48790,461400,not available,236053,23135,not available,not available
1,2020-01-02,วันพหัสบดี,1,normal,normal,65090,720500,not available,341059,39445,not available,not available
2,2020-01-03,วันศุกร์,1,normal,normal,69465,782100,not available,373005,43185,not available,not available
3,2020-01-04,วันเสาร์,0,normal,normal,53304,574400,not available,274151,30550,not available,not available
4,2020-01-05,วันอาทิตย์,0,normal,normal,50992,503000,not available,237162,26012,not available,not available


In [25]:
df_pivot.to_csv('../data/data_adjustmented.csv', index=False)